In [8]:
import torch
import torch.nn as nn 
import pandas as pd
from torch.utils.data import DataLoader
import numpy as np
from sklearn.model_selection import train_test_split



#from model import MILK_FFMixer_2stage
from Fake_Model import Fake_Model
from utils import CustomDataset


data_dir = 'input_example'
label_file = 'target_example.csv'

df_label = pd.read_csv(label_file, header=None, index_col=0)

images = [os.path.join(data_dir, f'{i}.npy') for i in df_label.index.to_list()]
labels = df_label.to_numpy(dtype=np.int64).flatten()

# Train/Valid/Test split
seed = 0
from monai.transforms import Compose, RandRotate

train_test_split_ratio = 0.8
train_val_split_ratio = 0.75
train_images, test_images, train_labels, test_labels = train_test_split(
    images, labels, train_size=train_test_split_ratio, random_state=seed,
    stratify=labels if len(labels) * (1 - train_test_split_ratio) > 1 else None,
)
train_images, val_images, train_labels, val_labels = train_test_split(
    train_images, train_labels, train_size=train_val_split_ratio, random_state=seed,
    stratify=train_labels if len(train_labels) * (1 - train_val_split_ratio) > 1 else None,
)

# Define transforms
train_transforms = Compose([RandRotate(range_x=10, range_y=10, range_z=10)])
val_transforms = None
test_transforms = None

# create a training data loader
trainset = CustomDataset(image_files=train_images, labels=train_labels, transform=train_transforms)
validset = CustomDataset(image_files=val_images, labels=val_labels, transform=val_transforms)
testset = CustomDataset(image_files=test_images, labels=test_labels, transform=test_transforms)

##############################################################################################################################

print('DataLoader Generation')



train_sampler = None
valid_sampler = None
test_sampler  = None
suffle=False

train_loader = DataLoader(trainset, num_workers=0, shuffle=False,
                      sampler=train_sampler, batch_size=4 ,
                      pin_memory=False, drop_last=False,
                      collate_fn=None )   

DataLoader Generation


In [21]:
class Fake_Model(nn.Module):
    def __init__(self):
        super(Fake_Model, self).__init__()
        
    def forward(self, inputs, DEBUG=False):
        out = inputs[:,0,0,0,0]
        return out 
    
    def infer(self, inputs, DEBUG=False):    
        return self.forward(inputs, DEBUG=DEBUG)
model = Fake_Model()

In [15]:
next(iter(train_loader))[0].shape

torch.Size([3, 1, 32, 224, 224])

In [16]:
next(iter(train_loader))[1].shape

torch.Size([3])

In [22]:
model(next(iter(train_loader))[0]).shape

torch.Size([3])

In [3]:
import os
os.getenv('WORLD_SIZE', 1)

1

In [25]:
torch.tensor(1).cast(next(iter(train_loader))[0])

AttributeError: 'Tensor' object has no attribute 'cast'

In [1]:
import torch
 
print("쿠다 가능 :{}".format(torch.cuda.is_available()))
print("현재 디바이스 :{}".format(torch.cuda.current_device() ))
print("디바이스 갯수 :{}".format(torch.cuda.device_count()))
for idx in range(0, torch.cuda.device_count()  ):
    print("디바이스 :{}".format(torch.cuda.device(idx)))
    print("디바이스 이름 :{}".format(torch.cuda.get_device_name(idx)))

쿠다 가능 :True
현재 디바이스 :0
디바이스 갯수 :2
디바이스 :<torch.cuda.device object at 0x7f1a44138cd0>
디바이스 이름 :NVIDIA GeForce RTX 3090
디바이스 :<torch.cuda.device object at 0x7f1a44138280>
디바이스 이름 :NVIDIA GeForce RTX 3090
